In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 57.6 MB/s 
     |████████████████████████████████| 163 kB 89.6 MB/s 


In [2]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import re

## 데이터

In [4]:
data = pd.concat([pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_10000.csv'),
                  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_20000.csv'),
                  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_30000.csv'),
                  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_40000.csv'),
                  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_50000.csv'),
                  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_60000.csv'),
                  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_70000.csv'),
                  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/talk_74665.csv')])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,4

In [5]:
data.reset_index(drop = True, inplace = True)
data.head()

,talk_id,0,1,2,3,4,5,6,7,8,...,9907,9908,9909,9910,9911,9912,9913,9914,9915,9916
0,MDRW2100000001.1,NaN,안녕하세요,️,이거 해봐요><,"나의 직장인 멘탈 성향은 [안챙겨도 잘커요, 탕비실 선인장] 당신의 멘탈 성향은...",오 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 안챙겨도 잘커요 너무 맞는데요ㅜㅜ? 자세한 내용은 더 알아...,ㅋㅌㅋㅋㅋㅋ 아녜여 챙겨주세요,ㅋㅋㅋㅋㅋ당연하쥬 누굴 챙길 여유는 저도 없는거같지만 그러나 점심 메뉴는 언제든 ...,그렇게 큰 권한을 주신다구요?? name1님 완전 대인배,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MDRW2100000001.2,NaN,"목요일 점심메뉴도 생각해오세요 크크 전 닭가슴살 먹을거지만,,",흠 그럼 저도 한번 도시락을 싸올까요,ㅋㅋㅋㅋㅋㅋㅋㅋ 도시락쌀 기운이 있다싶으면 좋아요~~ 부지런한편이에요?,ㅋㅋㅋㅋ저요? 저 완전 발발이에요,mbti p는 게으르잖아요 부지런한 p인건가 ㅋㅋㅋㅋㅋㅋㅋㅋ,맞아요 그래서 일을 벌려놔요 일을 저지르면 해결해야해서,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 오 멋있다 그래도,여기 면접본것도 사실 일단 저질러서...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MDRW2100000001.3,NaN,전 용두사미 스타일~~ 일단 호기롭게 시작하고 끝이 미미해.. 그냥 해치우자! 느...,ㅋㅋㅋㅋ그래도 시작이 반이잖아요?!,저도 그렇게 생각해요 뭐라도 해보는게 낫지!! ㅋㅋㅋㅋㅋㅋㅋ,맞아요! 가만히 있는것보다야 훨 낫죠,맞아요 ! 뭐라도 얻는게 있겠죠... 헿 내일 잘쉬어요 ㅜㅜ 목요일에 엉덩이\r\...,name1님은.. 재밌게... 다녀오세요 제 엉덩이.....,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,잘 말려서 올게요,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MDRW2100000001.4,NaN,서서일하는것도 추천 ㅎㅎ,ㅋㅌㅋㅋㅌㅌㅋㅋ앟 그럼 너무 튀어요 안그래도 name3팀장님이 쳐다보는거 느껴진단...,아까 심리테스트에 관종이라고 적혀있던데...? ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 관심 크게 ...,ㅋㅌㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 아녜요.. 교수님이랑 친구들이 직장에선 자제하라고 했...,아까 채팅에서 이미 조금 드러났어요... ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,언제요??? 진ㅁ자요???,전 많이요...할때 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MDRW2100000002.1,NaN,하이하이,반가워욬ㅌㅋㅋ name2님 제 이상형은 코가 예쁘면 일단 외관 통관데 name2님...,오 저는 무조건 무쌍 존잘이여 아니 다들 내 이상형 못생겻다구우기는데 무쌍 존잘이...,ㅋㅌㅋ근데 무쌍 존잘 현실에 별로 없잖아요....,요새 강철부대 육준서에 마음이 선덕선덕댑니다.^^ 아 인정 저는 투디로 행복하려구요 ~,ㅌㅌㅋㅋ앗 그분은 2.5d 잖아요,"투디 체고! ! ! 차피 이상형 못만남,, ㅋㅋㅋㅋㅋㅋ인정,, 무쌍 체고야 짜릿해",헐 방금 찾아봤는데 개존잘이네요,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AURA2/data/main/Speaker_info.csv')

In [7]:
info.head()

,talk_id,topic,p1_id,p1_age,p1_occ,p1_sex,p1_birthplace,p1_residence,p2_id,p2_age,p2_occ,p2_sex,p2_birthplace,p2_residence,relation,inti
0,MDRW2100000001.1,일상 트렌드,1,20대,전문가 및 관련 종사자,여성,울산,경기,2,20대,사무 종사자,여성,서울,서울,직장>동기/동료/동업자,5
1,MDRW2100000001.2,식음료,1,20대,전문가 및 관련 종사자,여성,울산,경기,2,20대,사무 종사자,여성,서울,서울,직장>동기/동료/동업자,5
2,MDRW2100000001.3,일과 직업,1,20대,전문가 및 관련 종사자,여성,울산,경기,2,20대,사무 종사자,여성,서울,서울,직장>동기/동료/동업자,5
3,MDRW2100000001.4,일과 직업,1,20대,전문가 및 관련 종사자,여성,울산,경기,2,20대,사무 종사자,여성,서울,서울,직장>동기/동료/동업자,5
4,MDRW2100000002.1,연애와 결혼,1,20대,사무 종사자,여성,서울,서울,2,20대,전문가 및 관련 종사자,여성,서울,서울,직장>선후배/상사-부하,3


In [8]:
int(info[info['talk_id'] == re.sub(' ', '', 'MDRW2100000001.1')]['inti'][0])

5

In [9]:
# [CLS] [SEP] 넣고 합치기
sentences = []
labels = []
for i in range(len(data)) :
  id = re.sub(' ', '', data.loc[i, 'talk_id'])
  inti_arr = (info[info['talk_id'] == id]['inti'])
  if len(inti_arr) != 1 :
    continue
  else :
    inti = int(inti_arr)
  if inti != 5 and inti != 0 and inti != 4:
    continue 
  sentence = ''
  for j in range(1, int(data.columns[-1])+1) :
    #if len(re.sub(' ', '', data.loc[i, str(j)])) == 0 :
    #  continue
    if pd.isnull(data.loc[i, str(j)]) :
      break
    if type(data.loc[i, str(j)]) == float :
      continue
    sent = data.loc[i, str(j)]
    #sent = re.sub('name[0-9]+', '', sent) # name 삭제
    sent = re.sub('[^ㄱ-힣 ]', '', sent)
    sent = re.sub('[ㄱ-ㅎ]', '', sent) # ㅋㅋㅋ ㅎㅎㅎ 삭제
    sent = re.sub('[ㅏ-ㅣ]', '', sent) # 모음 삭제
    #sent=  re.sub('account', '', sent) # account 삭제
    #sent=  re.sub('others', '', sent) # others 삭제
    #sent = re.sub('[.]', '', sent) # .삭제
    #sent = re.sub('[,]', '', sent) # ,삭제
    #sent = re.sub('\r', '', sent) # \r삭제
    #sent = re.sub('\n', '', sent) # \n삭제
    if j == 1 :
      sentence += ('[CLS]' + sent + '[SEP]')
    else :
      sentence += (sent + '[SEP]')
    
  sentences.append(sentence)
  if inti == 5 or inti == 4 :
    labels.append(1)
  else :
    labels.append(0)

In [10]:
sentences[:5]

['[CLS] 안녕하세요[SEP] [SEP]  이거 해봐요[SEP] 나의 직장인 멘탈 성향은  안챙겨도 잘커요 탕비실 선인장 당신의 멘탈 성향은 어떠신가요 지금 테스트해보세요  아앗[SEP] 오  안챙겨도 잘커요 너무 맞는데요 자세한 내용은 더 알아가야겟지만[SEP]  아녜여 챙겨주세요[SEP] 당연하쥬 누굴 챙길 여유는 저도 없는거같지만 그러나 점심 메뉴는 언제든 권한을 드릴게여 [SEP] 그렇게 큰 권한을 주신다구요 님 완전 대인배[SEP]',
 '[CLS] 목요일 점심메뉴도 생각해오세요 크크 전 닭가슴살 먹을거지만[SEP] 흠 그럼 저도 한번 도시락을 싸올까요[SEP]  도시락쌀 기운이 있다싶으면 좋아요 부지런한편이에요[SEP] 저요 저 완전 발발이에요[SEP]  는 게으르잖아요 부지런한 인건가 [SEP] 맞아요 그래서 일을 벌려놔요 일을 저지르면 해결해야해서[SEP]  오 멋있다 그래도[SEP] 여기 면접본것도 사실 일단 저질러서[SEP]  근데 저도[SEP]',
 '[CLS] 전 용두사미 스타일 일단 호기롭게 시작하고 끝이 미미해 그냥 해치우자 느낌 [SEP] 그래도 시작이 반이잖아요[SEP] 저도 그렇게 생각해요 뭐라도 해보는게 낫지 [SEP] 맞아요 가만히 있는것보다야 훨 낫죠[SEP] 맞아요  뭐라도 얻는게 있겠죠 헿 내일 잘쉬어요  목요일에 엉덩이지켜야지[SEP] 님은 재밌게 다녀오세요 제 엉덩이[SEP] [SEP] 잘 말려서 올게요[SEP]',
 '[CLS] 서서일하는것도 추천  [SEP] 앟 그럼 너무 튀어요 안그래도 팀장님이 쳐다보는거 느껴진단말이에여[SEP] 아까 심리테스트에 관종이라고 적혀있던데  관심 크게 한번 받으세여[SEP]  아녜요 교수님이랑 친구들이 직장에선 자제하라고 했어여[SEP] 아까 채팅에서 이미 조금 드러났어요 [SEP] 언제요 진자요[SEP] 전 많이요할때 [SEP]',
 '[CLS] 초밥 맛있겠다 [SEP] 너 초밥 먹어[SEP] 웅[SEP] 마싯겟당[SEP] 너도 먹을거자나[SEP] 나두 초밥먹을까 아냐 난 아직

In [11]:
len(sentences) == len(labels)

True

In [12]:
label_count = {'0' : 0, '1' : 0}
for i in labels:
  label_count[str(i)] += 1
print(label_count['0'], label_count['1'])

37044 23482


In [13]:
# 토크나이징
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case = False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print(sentences[0])
print(tokenized_texts[0])

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

[CLS] 안녕하세요[SEP] [SEP]  이거 해봐요[SEP] 나의 직장인 멘탈 성향은  안챙겨도 잘커요 탕비실 선인장 당신의 멘탈 성향은 어떠신가요 지금 테스트해보세요  아앗[SEP] 오  안챙겨도 잘커요 너무 맞는데요 자세한 내용은 더 알아가야겟지만[SEP]  아녜여 챙겨주세요[SEP] 당연하쥬 누굴 챙길 여유는 저도 없는거같지만 그러나 점심 메뉴는 언제든 권한을 드릴게여 [SEP] 그렇게 큰 권한을 주신다구요 님 완전 대인배[SEP]
['[CLS]', '안', '##녕', '##하', '##세', '##요', '[SEP]', '[SEP]', '이', '##거', '해', '##봐', '##요', '[SEP]', '나', '##의', '직', '##장', '##인', '멘', '##탈', '성', '##향', '##은', '안', '##챙', '##겨', '##도', '잘', '##커', '##요', '탕', '##비', '##실', '선', '##인', '##장', '당', '##신의', '멘', '##탈', '성', '##향', '##은', '어', '##떠', '##신', '##가', '##요', '지', '##금', '테', '##스트', '##해', '##보', '##세', '##요', '아', '##앗', '[SEP]', '오', '안', '##챙', '##겨', '##도', '잘', '##커', '##요', '너', '##무', '맞', '##는데', '##요', '자', '##세', '##한', '내', '##용', '##은', '더', '알', '##아', '##가', '##야', '##겟', '##지만', '[SEP]', '[UNK]', '챙', '##겨', '##주', '##세', '##요', '[SEP]', '당', '##연', '##하', '##쥬', '누', '##굴', '챙', '##길', '여', '##유', '##는', '저', '##도', '없는', '##거', '##같', '##지만', '그러나', '점', '##심', 

In [14]:
max = max(len(t) for t in tokenized_texts)
avg = sum(len(t) for t in tokenized_texts) / len(tokenized_texts)

In [15]:
print('Max : ', max)
print('Avg : ', avg)

Max :  108173
Avg :  344.89824207778474


In [16]:
MAX_LEN = 256
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

input_ids[0]

array([   101,   9521, 118741,  35506,  24982,  48549,    102,    102,
         9638,  41521,   9960, 118990,  48549,    102,   8982,  10459,
         9707,  13890,  12030,   9274, 119336,   9434,  79544,  10892,
         9521, 119258,  89045,  12092,   9654, 106826,  48549,   9853,
        29455,  31503,   9428,  12030,  13890,   9067,  87143,   9274,
       119336,   9434,  79544,  10892,   9546, 118833,  25387,  11287,
        48549,   9706,  40032,   9866,  34994,  14523,  30005,  24982,
        48549,   9519, 119117,    102,   9580,   9521, 119258,  89045,
        12092,   9654, 106826,  48549,   9004,  32537,   9256,  41850,
        48549,   9651,  24982,  11102,   8996,  24974,  10892,   9074,
         9524,  16985,  11287,  21711, 118631,  28578,    102,    100,
         9743,  89045,  16323,  24982,  48549,    102,   9067,  25486,
        35506, 119224,   9032, 118654,   9743, 118666,   9565,  42815,
        11018,   9663,  12092,  40364,  41521, 118615,  28578,  21890,
      

In [17]:
attention_masks = []

for seq in input_ids :
  seq_masks = [float(i>0) for i in seq]
  attention_masks.append(seq_masks)
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [18]:
# 분리
train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_ids, labels, random_state = 2022, test_size = 0.1)

train_masks, test_masks, _, _ = train_test_split(attention_masks, input_ids, random_state = 2022, test_size = 0.1)

train_inputs_, validation_inputs, train_labels, validation_labels = train_test_split(train_inputs, train_labels, random_state = 2022, test_size = 0.1)

train_masks, validation_masks, _, _ = train_test_split(train_masks, train_inputs, random_state = 2022, test_size = 0.1)

train_inputs = torch.tensor(train_inputs_)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
test_inputs = torch.tensor(test_inputs)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])
print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   8982,   9638,  35465,  10530,  67778,  14279,   9708,  23321,
         14523,    102,  82564,   9953,  11882,   9638,  25503,    102,   8996,
         11287,   9802, 119410,  21876,   9685,  16985,  12178,   8863,   9524,
         12508,    102,   9067,  25486,  18108,   9524,  12508,    102,   9559,
          9638,   9519,  10739,   9953,  11882,  11513,   8843,  11664,   9495,
         12965,    102,   9640,  28000,  12508,  74986,  10739,   9640,  12310,
         59894,  11018,   9117, 119138,  12965,    102,   9256,  16985,   9604,
         12692,   9953,  25242,   9640,  28000,  12508,  74986,   9087,   8906,
         13890,  18108,   9625,  16758,  14523,    102,   8924, 118866,   8885,
        119202,  88350,  10739,   9028, 118632,  41850,    102,   8996,  11287,
          9953,  34907,  10739,   9028,  12508,   9523,  16985,  12424,   8888,
         36553,  10739,  21711,    102,   9640,  28000,  12508,  74986,   9953,
         11882,  11287,   9611, 118714, 

In [19]:
batch_size = 64

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler = validation_sampler, batch_size = batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size = batch_size)

## 모델 생성 및 학습

In [20]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [21]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: A100-SXM4-40GB


In [22]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [23]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [24]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [25]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [26]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()


# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 출력 로짓 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of    767.    Elapsed: 0:05:17.

  Average training loss: 0.21
  Training epcoh took: 0:08:04

Running Validation...
  Accuracy: 0.94
  Validation took: 0:00:17

======== Epoch 2 / 4 ========
Training...
  Batch   500  of    767.    Elapsed: 0:05:14.

  Average training loss: 0.16
  Training epcoh took: 0:08:01

Running Validation...
  Accuracy: 0.94
  Validation took: 0:00:17

======== Epoch 3 / 4 ========
Training...
  Batch   500  of    767.    Elapsed: 0:05:14.

  Average training loss: 0.13
  Training epcoh took: 0:08:01

Running Validation...
  Accuracy: 0.95
  Validation took: 0:00:17

======== Epoch 4 / 4 ========
Training...
  Batch   500  of    767.    Elapsed: 0:05:14.

  Average training loss: 0.11
  Training epcoh took: 0:08:01

Running Validation...
  Accuracy: 0.94
  Validation took: 0:00:17

Training complete!


## 모델 테스트

In [27]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 출력 로짓 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.94
Test took: 0:00:19
